<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/chat_engine/chat_engine_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Chat Engine - Condense Plus Context Mode

This is a multi-step chat mode built on top of a retriever over your data.

For each chat interaction:
* First condense a conversation and latest user message to a standalone question
* Then build a context for the standalone question from a retriever,
* Then pass the context along with prompt and user message to LLM to generate a response.

This approach is simple, and works for questions directly related to the knowledge base and general interactions.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

## Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2023-11-16 12:12:20--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: 'data/paul_graham/paul_graham_essay.txt'

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.05s   

2023-11-16 12:12:29 (1.39 MB/s) - 'data/paul_graham/paul_graham_essay.txt' saved [75042/75042]



## Get started in 5 lines of code

Load data and build index

In [ ]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI


service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo")
)
data = SimpleDirectoryReader(input_dir="./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(data, service_context=service_context)

Configure chat engine

Since the context retrieved can take up a large amount of the available LLM context, let's ensure we configure a smaller limit to the chat history!

In [ ]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Based on the above documents, provide a detailed answer for the user question below."
    ),
    verbose=True,
)

Chat with your data

In [ ]:
response = chat_engine.chat("Who are you?")

In [ ]:
print(response)

I am an AI chatbot designed to assist and engage in conversations with users. I can provide information, answer questions, and engage in discussions on various topics. Is there anything specific you would like to know or discuss?


Ask a follow up question

In [ ]:
response_2 = chat_engine.chat("What did Paul Graham do growing up?")

In [ ]:
print(response_2)

Growing up, Paul Graham had two main interests: writing and programming. He initially wrote short stories, although he admits that they were not very good. At the age of 13 or 14, he started programming on an IBM 1401 computer that his school district used for data processing. However, he found it challenging to figure out what to do with the computer since the only input option was punched cards, and he didn't have any data stored on them. Despite this, he continued exploring programming.

When microcomputers became available, Graham's interest in programming grew. He convinced his father to buy a TRS-80 computer, and that's when he really started programming. He wrote simple games, a program to predict rocket heights, and even a word processor that his father used to write a book. This experience with microcomputers allowed him to have a computer right in front of him, responding to his keystrokes, which was a significant change from the punch card system.

Overall, Graham's early ex

In [ ]:
response = chat_engine.chat("Can you tell me more?")

In [ ]:
print(response)

Certainly! In the essay discussing Paul Graham's life, he talks about his early experiences with writing and programming. Before college, he wrote short stories, although he admits they were not very good. At the same time, he started programming on an IBM 1401 computer in his school's basement. However, he found it challenging to figure out what to do with the computer since the only input was data stored on punched cards.

Graham's interest in programming grew when microcomputers became available. He was impressed by a friend who built his own microcomputer from a kit. Eventually, Graham convinced his father to buy a TRS-80 computer, which he used to write simple games, a rocket prediction program, and even a word processor that his father used to write a book.

Despite his growing passion for programming, Graham initially planned to study philosophy in college. He believed that philosophy dealt with ultimate truths compared to other fields that focused on domain knowledge. However, 

Reset conversation state

In [ ]:
chat_engine.reset()

In [ ]:
response = chat_engine.chat("Hello! What do you know?")

In [ ]:
print(response)

Hello! As an AI chatbot, I have access to a wide range of information. I can provide general knowledge, answer questions, engage in conversations, and assist with various topics. Is there something specific you would like to know or discuss?


## Streaming Support

In [ ]:
from llama_index import (
    ServiceContext,
    VectorStoreIndex,
    SimpleDirectoryReader,
    set_global_service_context,
)
from llama_index.llms import OpenAI

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)
set_global_service_context(service_context)

data = SimpleDirectoryReader(input_dir="./data/paul_graham/").load_data()

index = VectorStoreIndex.from_documents(data)

In [ ]:
chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Based on the above documents, provide a detailed answer for the user question below."
    ),
)

In [ ]:
response = chat_engine.stream_chat("What did Paul Graham do after YC?")
for token in response.response_gen:
    print(token, end="")

After Y Combinator (YC), Paul Graham made a significant decision to step back from his role in YC and pursue other endeavors. In 2012, his mother had a stroke caused by colon cancer, which led him to reevaluate his priorities. He realized that YC was consuming more of his attention and that he was ready to hand over the reins to someone else.

Paul approached Jessica Livingston, his wife and co-founder of YC, to take over as president, but she declined. Eventually, they recruited Sam Altman, who initially wanted to start a startup focused on nuclear reactors. However, Paul persisted in convincing Sam to join YC, and in October 2013, Sam agreed to become the president of YC.

During the transition period, Paul gradually handed over the responsibilities of running YC to Sam, allowing him to learn the job. This allowed Paul to focus on his mother, who was battling cancer. Ultimately, Paul retired from his active role in YC, along with co-founder Robert Morris, while Jessica Livingston and